In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

agentId = os.getenv('BEDROCK_AGENT_ID')
agentAliasId = os.getenv('BEDROCK_AGENT_ALIAS_ID')
region_name = 'us-east-1'
lambda_function_arn = os.getenv('LAMBDA_FUNCTION_ARN')

In [2]:
import boto3
import uuid
from helper import *

In [3]:
sessionId = str(uuid.uuid4())
message = "My name is Abhishek, the iphone i recieved is defective and I want a refund."

In [ ]:
invoke_agent_and_print(
    agentId=agentId, 
    agentAliasId=agentAliasId, 
    inputText=message, 
    sessionId=sessionId
)

In [5]:
bedrock_agent= boto3.client(service_name = 'bedrock-agent', region_name = region_name)

In [8]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName="customer-support-for-e-commerce-platform",
    agentId=agentId,
    actionGroupExecutor = {
        "lambda": lambda_function_arn
    },
    functionSchema={
        "functions":[
            {
                "name":"resolveCustomerId",
                "description": "Retrieve the customer ID based on available information.Private data is not to be shown to user.",
                "parameters":{
                    "email":{
                        "description": "email address of the customer",
                        "required": False,
                        "type":"string"
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'escalateToSupport',
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    }
                }
            },
            {
                'name': 'getTicketStatus',
                'description': 'Check the status of an existing support ticket.',
                'parameters': {
                    'ticketId': {
                        'description': 'Ticket ID to check',
                        'required': True,
                        'type': 'string'
                    }
                }
            },
{
    'name': 'getOrderStatus',
    'description': 'Retrieve the status of a customer order using order ID.',
    'parameters': {
        'orderId': {
            'description': 'Unique order identifier',
            'required': True,
            'type': 'string'
        }
    }
},
{
    'name': 'cancelSubscription',
    'description': 'Cancel a customer subscription. Use only upon verified request.',
    'parameters': {
        'custId': {
            'description': 'Customer ID of the account',
            'required': True,
            'type': 'string'
        },
        'reason': {
            'description': 'Reason for cancellation',
            'required': False,
            'type': 'string'
        }
    }
}

        ]
    },
    agentVersion='DRAFT',
)
                

In [9]:
create_agent_action_group_response

{'ResponseMetadata': {'RequestId': 'cd3462da-c481-4e70-be47-29c0fc2069dd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 04 Aug 2025 08:58:43 GMT',
   'content-type': 'application/json',
   'content-length': '1937',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cd3462da-c481-4e70-be47-29c0fc2069dd',
   'x-amz-apigw-id': 'OxdqjH-kIAMEm7w=',
   'x-amzn-trace-id': 'Root=1-68907643-687aa9b6600891d6734e1c34'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:038145523842:function:lambdaformyagent'},
  'actionGroupId': 'GLB9M3V4SX',
  'actionGroupName': 'customer-support-for-e-commerce-platform',
  'actionGroupState': 'ENABLED',
  'agentId': 'NNCSQ7FM3I',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2025, 8, 4, 8, 58, 43, 227996, tzinfo=tzutc()),
  'functionSchema': {'functions': [{'description': 'Retrieve the customer ID based on available information.Private data is not to be shown to user.',
     'n

In [10]:
actionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']


In [11]:
wait_for_action_group_status(
    agentId=agentId,
    actionGroupId=actionGroupId,
    targetStatus="ENABLED"
)

Action Group status: ENABLED


'ENABLED'

In [12]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [13]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='Aliasformyagent',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [14]:
sessionId = str(uuid.uuid4())
message = "hey my name is x with email of  x@x.xom, my iphone was broken"

In [16]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=False
)

User: hey my name is x with email of  x@x.xom, my iphone was broken

Agent: I have escalated your support request for your broken iPhone to our
       support team. They have assigned a support ticket with ID 3613
       to address your issue. You can check the status of this ticket
       by using the '<REDACTED>' function and providing the ticket ID.
       Please let me know if you have any other questions!

Session ID: 6aad4322-90e5-4330-bb7f-6d1728ff257e


In [17]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: hey my name is x with email of  x@x.xom, my iphone was broken

Agent: 
Agent's thought process:
  It looks like you are reporting the same issue with your broken
  iPhone again. Let me try to assist you further.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-for-e-commerce-platform
  Function: resolveCustomerId
  Parameters: [{'name': 'name', 'type': 'string', 'value': 'x'}, {'name': 'email', 'type': 'string', 'value': 'x@x.xom'}]

Observation:
  Type: ACTION_GROUP
  Action Group Output: {'id':8935}

Agent's thought process:
  Okay, I was able to retrieve your customer ID as 8935 based on the
  name and email address you provided. Let me go ahead and escalate
  your support request for the broken iPhone again.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-for-e-commerce-platform
  Function: escalateToSupport
  Parameters: [{'name': 'supportSummary', 'type': 'string', 'value': "Customer's iPhone is broken"}, {'name': 'custId', '

In [19]:
message = "i dont like the service , i want to cancel my paid subscription"

invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: i dont like the service , i want to cancel my paid subscription

Agent: 
Agent's thought process:
  Okay, let me see how I can assist you with cancelling your paid
  subscription. I will first need to retrieve your customer ID to
  process the cancellation request.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-for-e-commerce-platform
  Function: resolveCustomerId
  Parameters: [{'name': 'name', 'type': 'string', 'value': 'x'}, {'name': 'email', 'type': 'string', 'value': 'x@x.xom'}]

Observation:
  Type: ACTION_GROUP
  Action Group Output: {'id':4152}

Agent's thought process:
  Great, I was able to retrieve your customer ID as 4152 based on the
  information you provided. Now I can proceed with cancelling your
  paid subscription.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-for-e-commerce-platform
  Function: cancelSubscription
  Parameters: [{'name': 'reason', 'type': 'string', 'value': 'Customer is not satisfied with the 

In [20]:
sessionId = str(uuid.uuid4())
message = "ihey what s the status of my order number 5678"

invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: ihey what s the status of my order number 5678

Agent: 
Agent's thought process:
  To check the status of the customer's order, I will need to invoke
  the 'getOrderStatus' function and provide the order ID.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-for-e-commerce-platform
  Function: getOrderStatus
  Parameters: [{'name': 'orderId', 'type': 'string', 'value': '5678'}]

Observation:
  Type: ACTION_GROUP
  Action Group Output: {'orderId': '5678', 'status': 'Shipped'}

Observation:
  Type: FINISH

Final response:
  According to the order status, your order number 5678 has been
  Shipped.


Session ID: a0f26a01-bf1f-4f0d-83f6-87c8d977009b
